In [1]:
import os
import warnings
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

warnings.filterwarnings("ignore")


In [2]:
print("📍 Current working directory:", os.getcwd())

DATA_PATH = "crypto_master_cleaned.csv"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ File not found: {os.path.abspath(DATA_PATH)}")

df = pd.read_csv(DATA_PATH)

# Normalize column names (IMPORTANT)
df.columns = df.columns.str.strip().str.lower()

# Convert date
df["date"] = pd.to_datetime(df["date"])

df.head()


📍 Current working directory: D:\amdox technology\Crypto time series analysis\notebooks


,date,open_price,high_price,low_price,close_price,volume,crypto
0,2020-12-31,0.184034,0.185749,0.176594,0.181397,1132268397,ADA
1,2021-01-01,0.181382,0.184246,0.172022,0.175350,1122218004,ADA
2,2021-01-02,0.175359,0.184253,0.169233,0.177423,1408849504,ADA
3,2021-01-03,0.177382,0.208679,0.173376,0.204995,2303857909,ADA
4,2021-01-04,0.205236,0.239661,0.194450,0.224762,3260699086,ADA


In [3]:
print(df.columns)
print(df["crypto"].unique())
print(df.isna().sum())


Index(['date', 'open_price', 'high_price', 'low_price', 'close_price',
       'volume', 'crypto'],
      dtype='str')
<StringArray>
['ADA', 'BCH', 'BNB', 'BTC', 'DOGE', 'DOT', 'ETH', 'LTC', 'SOL', 'XRP']
Length: 10, dtype: str
date           0
open_price     0
high_price     0
low_price      0
close_price    0
volume         0
crypto         0
dtype: int64


In [4]:
FORECAST_STEPS = 30

arima_rows = []
sarima_rows = []

coins = df["crypto"].unique()

for coin in coins:
    print(f"🔮 Forecasting {coin}")

    coin_df = df[df["crypto"] == coin].sort_values("date")
    ts = coin_df.set_index("date")["close_price"]

    # ---------- ARIMA ----------
    try:
        arima_model = ARIMA(ts, order=(1, 1, 1))
        arima_fit = arima_model.fit()
        arima_forecast = arima_fit.forecast(steps=FORECAST_STEPS)

        for d, val in arima_forecast.items():
            arima_rows.append({
                "date": d,
                "crypto": coin,
                "forecast_price": float(val),
                "model": "ARIMA"
            })
    except Exception as e:
        print(f"❌ ARIMA failed for {coin}: {e}")

    # ---------- SARIMA ----------
    try:
        sarima_model = SARIMAX(
            ts,
            order=(1, 1, 1),
            seasonal_order=(1, 1, 1, 12),
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        sarima_fit = sarima_model.fit(disp=False)
        sarima_forecast = sarima_fit.forecast(steps=FORECAST_STEPS)

        for d, val in sarima_forecast.items():
            sarima_rows.append({
                "date": d,
                "crypto": coin,
                "forecast_price": float(val),
                "model": "SARIMA"
            })
    except Exception as e:
        print(f"❌ SARIMA failed for {coin}: {e}")


🔮 Forecasting ADA
🔮 Forecasting BCH
🔮 Forecasting BNB
🔮 Forecasting BTC
🔮 Forecasting DOGE
🔮 Forecasting DOT
🔮 Forecasting ETH
🔮 Forecasting LTC
🔮 Forecasting SOL
🔮 Forecasting XRP


In [5]:
ARIMA_PATH = "arima_all_coins.csv"

if not os.path.exists(ARIMA_PATH):
    raise FileNotFoundError(f"❌ File not found: {ARIMA_PATH}")

arima_df = pd.read_csv(ARIMA_PATH)
arima_df["date"] = pd.to_datetime(arima_df["date"])

print("✅ ARIMA data loaded")



✅ ARIMA data loaded


In [6]:
arima_df.head()

,date,crypto,forecast_price,model
0,1970-01-01 00:00:00.000000429,ADA,0.835739,ARIMA
1,1970-01-01 00:00:00.000000430,ADA,0.833490,ARIMA
2,1970-01-01 00:00:00.000000431,ADA,0.835139,ARIMA
3,1970-01-01 00:00:00.000000432,ADA,0.833930,ARIMA
4,1970-01-01 00:00:00.000000433,ADA,0.834816,ARIMA


In [7]:
arima_df.tail()

,date,crypto,forecast_price,model
295,1970-01-01 00:00:00.000000454,XRP,0.735948,ARIMA
296,1970-01-01 00:00:00.000000455,XRP,0.735948,ARIMA
297,1970-01-01 00:00:00.000000456,XRP,0.735948,ARIMA
298,1970-01-01 00:00:00.000000457,XRP,0.735948,ARIMA
299,1970-01-01 00:00:00.000000458,XRP,0.735948,ARIMA
